## WEB SCRAPING ##   

In [4]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

headers = {

  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'

}
response = r.get("https://www.imdb.com/pt/chart/top/?ref_=nv_mv_250", headers=headers)

soup= BeautifulSoup(response.content, 'html.parser')

## JSON ##

In [5]:
import json
df = soup.find('script', type='application/ld+json')

data = json.loads(df.string)

## ETL -TRANTAMENTO DOS DADOS ##

In [6]:
# Listas para armazenar os dados
tipo = []
name = []
alternateName = []
description = []
image = []
aggregateRating = [] 
contentRating = []
genre = []
duration = []

# Iterando sobre os filmes
for item in data['itemListElement']:
    filme = item['item']
    
    tipo.append(filme.get('@type', ''))
    name.append(filme.get('name', ''))
    alternateName.append(filme.get('alternateName', ''))
    description.append(filme.get('description', ''))
    image.append(filme.get('image', ''))
    aggregateRating.append(filme.get('aggregateRating', {}))  # Pode ser um dict
    contentRating.append(filme.get('contentRating', ''))
    genre.append(filme.get('genre', ''))
    duration.append(filme.get('duration', ''))

# Criando o DataFrame
filmes_250 = pd.DataFrame({
    'tipo': tipo,
    'name': name,
    'alternateName': alternateName,
    'description': description,
    'image': image,
    'aggregateRating': aggregateRating,
    'contentRating': contentRating,
    'genre': genre,
    'duration': duration
})


# Garante que só processe se for um dicionário
filmes_250['aggregate_type'] = filmes_250['aggregateRating'].apply(lambda x: x.get('@type') if isinstance(x, dict) else None)
filmes_250['bestRating'] = filmes_250['aggregateRating'].apply(lambda x: x.get('bestRating') if isinstance(x, dict) else None)
filmes_250['worstRating'] = filmes_250['aggregateRating'].apply(lambda x: x.get('worstRating') if isinstance(x, dict) else None)
filmes_250['ratingValue'] = filmes_250['aggregateRating'].apply(lambda x: x.get('ratingValue') if isinstance(x, dict) else None)
filmes_250['ratingCount'] = filmes_250['aggregateRating'].apply(lambda x: x.get('ratingCount') if isinstance(x, dict) else None)


In [9]:
filmes_250.shape

(250, 14)

In [ ]:
filmes_250